In [4]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
import os

genai.configure(
    api_key=os.environ['API_KEY'])
model_gemini = genai.GenerativeModel(
    model_name='gemini-pro')


c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# import docx2txt
import time
# import pandas
import docx
import re


# Function to apply gemini-3.5 model to a given text

In [6]:
def generate_text(work, rubrics):
    rubrics = rubrics
    text = work
    instructor = """an experienced language instructor and assess the work of students. 
    I will provide the student's essay and the rubrics. 
    Your task is to assign a score and provide detailed comments for each rubric. 
    Highlight words, sentences, or phrases where the student made mistakes or did not meet the rubric criteria."""

    prompt_aspect = f"""Act as {instructor} 
    Rubrics:
    {rubrics}

    Student Work: 
    {text}

    Instructions:
    1. Assign a score for each rubric criterion based on the student's essay.
    2. Provide detailed comments explaining the score for each criterion.
    3. Highlight specific mistakes or areas where the essay did not meet the rubric criteria.
    4. Offer constructive feedback for improvement where applicable."""

    try:
        completion = model_gemini.generate_content(
            prompt_aspect,
            generation_config={
                'temperature': 0.1,
                'max_output_tokens': 800
            }
        )
        score_gemini = completion.text
    except Exception as e:
        score_gemini = str(e)
    
    result = score_gemini
    return result


# Functions to read files

In [7]:
# Define function to read text from a given Word file
def read_word_file(filename):

    # Open the Word file
    doc = docx.Document(filename)

    # Extract the text from the Word file
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)

    # Print the text from the Word file
    return "\n".join(text)

# Define function to read text from a given PDF file
def read_pdf_file(filename):
    with open(filename, 'rb') as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text



# Set essay id and Folder location

# Read Rubrics and Descriptions

In [8]:
import PyPDF2

# Replace 'data_path' with the actual path to your folder containing the PDF
pdf_file = "Rubric_ Holistic Essay Scoring.pdf"

try:
  # Open the PDF file
  with open(pdf_file, 'rb') as pdf_obj:
      pdf_reader = PyPDF2.PdfReader(pdf_obj)

      # Get number of pages
      num_pages = len(pdf_reader.pages)
      print(f"The PDF has {num_pages} pages.")

      # Extract text (assuming it's a text PDF)
      text = ""
      for page_num in range(2,num_pages):
          page = pdf_reader.pages[page_num]
          text += page.extract_text()
      rubrics = text
      print(rubrics)

except Exception as e:
  print(f"An error occurred: {e}")


The PDF has 4 pages.
Holistic
Rating
for
Independent
Writing
After
reading
each
essay
and
completing
the
analytical
rating
form,
assign
a
holistic
score
based
on
the
rubric
below .
For
the
following
evaluations
you
will
need
to
use
a
grading
scale
between
1
(minimum)
and
6
(maximum).
As
with
the
analytical
rating
form,
the
distance
between
each
grade
(e.g.,
1-2,
3-4,
4-5)
should
be
considered
equal.
SCORE
OF
6:
An
essay
in
this
category
demonstrates
clear
and
consistent
mastery
,
although
it
may
have
a
few
minor
errors.
A
typical
essay
effectively
and
insightfully
develops
a
point
of
view
on
the
issue
and
demonstrates
outstanding
critical
thinking,
using
clearly
appropriate
examples,
reasons,
and
other
evidence
to
support
its
position;
the
essay
is
well
organized
and
clearly
focused,
demonstrating
clear
coherence
and
smooth
progression
of
ideas;
the
essay
exhibits
skillful
use
of
language,
using
a
varied,
accurate,
and
apt
vocabulary
and
demonstrates
meaningful
variety
in
sentence
stru

# Sample 30% DAtaset

In [36]:
import pandas as pd
data = pd.read_csv("train.csv")
# Step 1: Sample the first 30% of data from each unique 'score' group
sample_data = data.groupby('score').apply(lambda x: x.head(int(len(x) * 0.3))).reset_index(drop=True)
sample_data.to_csv('train_sample.csv', index = False)
class_counts = data['score'].value_counts()
sampled_class_counts = sample_data['score'].value_counts()

# Display the counts of samples in each class
print("length", len(sample_data), class_counts, sampled_class_counts)

C:\Users\Nimra\AppData\Local\Temp\ipykernel_15800\2378402893.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_data = data.groupby('score').apply(lambda x: x.head(int(len(x) * 0.3))).reset_index(drop=True)


length 5189 score
3    6280
2    4723
4    3926
1    1252
5     970
6     156
Name: count, dtype: int64 score
3    1884
2    1416
4    1177
1     375
5     291
6      46
Name: count, dtype: int64


# Call the process_files function

In [37]:

# Create a list of unique rubric names
unique_rubrics = ["Score"]
# Create a new dictionary for each unique rubric name to store the assessment results
rubric_results = {rubric: [] for rubric in unique_rubrics}
# df.head(5)
keywords = ["overall statement", "Comments", "Mistakes"]
results = {rubric: [] for rubric in keywords}
feedback= []


In [38]:
import pandas as pd
jump=2
if(os.path.isfile("results_gemini/Holistic_assessment_results_by_rubric.csv")):
    temp = pd.read_csv("results_gemini/Holistic_assessment_results_by_rubric.csv")
else:
    temp=[]
# Load the data from a CSV file
df = pd.read_csv("train_sample.csv")
print(len(df))
print(len(temp))
temp = temp[temp['essay_id'].isin(sample_data['essay_id'])]
temp.to_csv("results_gemini/Holistic_assessment_results_by_rubric.csv", index = False)
print(len(df) - len(temp))
df = df[~df['essay_id'].isin(temp['essay_id'])]
print("after slicing", len(df))
df = df.dropna(axis=1, how='all')
df.head(2)

5189
5189
0
after slicing 0


""


## Loop to Iterate through the CSV

In [39]:

for i in range(0, len(df), jump):
    data = df[i:i+jump] 
    # Iterate over the rows in the data
    for index, row in data.iterrows():

        # Get the text to assess
        text_to_assess = row["full_text"]    
        print(text_to_assess)
        # Apply the OpenAI model to the text to get the assessment result
        assessment_result = generate_text(work= text_to_assess, rubrics=rubrics)
        feedback.append(assessment_result)    
        print(assessment_result)
        # Extract the rubric values from the assessment result text
        for rubric in unique_rubrics:
            pattern = re.compile(rf"(?i){rubric}\D+(\d+(\.\d+)?)")
            match = pattern.search(assessment_result)
            print(match)
            if match:
                rubric_results[rubric].append(match.group(1))
            else:
                rubric_results[rubric].append("")
        for i in range(len(keywords)):
            keyword = keywords[i]
            pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
            match = pattern.search(assessment_result)
            if match:
                results[keyword].append(match.group(1).strip())
            else:
                results[keyword].append("")
        time.sleep(10)            
        print(rubric_results)

    for rubric in unique_rubrics:
        data[rubric+"_gemini"] = rubric_results[rubric]
    for keyword in keywords:
        data[keyword+"_gemini"] = results[keyword]
    data['Response_gemini'] = feedback
    # Save the data with the rubric columns to a new CSV file
    data.to_csv("results_gemini/Holistic_assessment_results_by_rubric.csv", index = False, mode='a', header=False)
    # data.to_csv("results_gemini/Holistic_assessment_results_by_rubric.csv", index = False)
    rubric_results = {rubric: [] for rubric in unique_rubrics}
    results = {rubric: [] for rubric in keywords}
    feedback=[]